# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.train.automl.run import AutoMLRun


from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

!pip install opendatasets

SDK version: 1.34.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:

import opendatasets as od
import time
import shutil

def download_data(url, dsname, downloadpath, file):
    if not os.path.exists(downloadpath):
        os.makedirs(downloadpath)

    od.download(url)
    data_downloadpath = os.path.join(dsname, file)
    data_finalpath = os.path.join(downloadpath, file)
    
    os.rename(data_downloadpath, data_finalpath)
    while not os.path.exists(data_finalpath):
        time.sleep(1)
    shutil.rmtree(f"./{dsname}")
    
    return data_finalpath

ws = Workspace.from_config()
dataurl = 'https://www.kaggle.com/rupeshraundal/marketcheck-automotive-data-us-canada/download'
kaggledsname = 'marketcheck-automotive-data-us-canada'
filenm = 'ca-dealers-used.csv'
downloadpath = 'Data'
dskey = 'UsedCarPrices'
dsdesc = 'Used Car Price Training Data'
found = False

dataset = None
if dskey in ws.datasets.keys(): 
    print(f"Retrieving registered dataset with key {dskey}")
    found = True
    dataset = ws.datasets[dskey] 

if not found:
    print("Downloading Canadian used car prices dataset from Kaggle")
    filepath = downloadpath
    
    if not os.path.exists(os.path.join(downloadpath, filenm)):
        datapath = download_data(dataurl, kaggledsname, downloadpath, filenm)
    
    
    # get the datastore to upload prepared data
    datastore = ws.get_default_datastore()
    
    
    print("Uploading to azure storage")
    # upload the local file from src_dir to the target_path in datastore
    datastore.upload(src_dir=downloadpath, target_path=downloadpath)
    print("Converting to azure dataset")
    dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, (os.path.join(filepath, filenm)))], separator=',', header=True, encoding='utf8')
    
    print("Registering dataset")
    dataset = dataset.register(workspace=ws,
                                 name=dskey,
                                 description=dsdesc)


print(f'Dataset ID: {dataset.id}')
print(f'Dataset name: {dataset.name}')
print(f'Dataset description: {dataset.description}')


Retrieving registered dataset with key UsedCarPrices
Dataset ID: 9927f2dd-0569-4f77-96fe-684f630609d6
Dataset name: UsedCarPrices
Dataset description: Used Car Price Training Data


In [4]:

print("Configuring compute cluster")
experiment_name = 'PCAP_Exp_CarPrices_AutoML'
experiment=Experiment(ws, experiment_name)


amlcompute_cluster_name = "PCAP-Clus-DS12v2"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes = 1, idle_seconds_before_scaledown = 120,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Configuring compute cluster
Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-automlstep-in-pipelines
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours": 0.5,
    "iteration_timeout_minutes":20,
    "enable_early_stopping": True,
    "early_stopping_n_iters": 5,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'r2_score',
    "verbosity": logging.INFO,
    "n_cross_validations": 5,
    "featurization":'auto',
    "model_explainability":True
}
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# Metrics like r2_score and spearman_correlation can better represent the quality of model when the scale of the value-to-predict covers many orders of magnitude. 
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "regression",
    training_data=dataset,
    label_column_name="price",  
    debug_log = "automl_errors.log",
    **automl_settings
)

In [15]:


# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=False)


Submitting remote run.
No run_configuration provided, running on PCAP-Clus-DS12v2 with default configuration
Running on remote compute: PCAP-Clus-DS12v2


Experiment,Id,Type,Status,Details Page,Docs Page
PCAP_Exp_CarPrices_AutoML,AutoML_2531e0e0-ceba-4864-b453-a94fe7cc18c5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
# from azureml.train.automl.run import AutoMLRun
# experiment = Experiment(ws, name="PCAP_Exp_CarPrices_AutoML")
# # remote_run = AutoMLRun(experiment, run_id = "AutoML_2531e0e0-ceba-4864-b453-a94fe7cc18c5")
# lastrun = experiment.get_runs(include_children=False)[0].id
# print(lastrun.id)
# remote_run = AutoMLRun(experiment, run_id = lastrun.id)
# print(remote_run)

Run(Experiment: PCAP_Exp_CarPrices_AutoML,
Id: AutoML_2531e0e0-ceba-4864-b453-a94fe7cc18c5,
Type: automl,
Status: Completed)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()



_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_2531e0e0-ceba-4864-b453-a94fe7cc18c5',
 'target': 'PCAP-Clus-DS12v2',
 'status': 'Completed',
 'startTimeUtc': '2021-10-03T20:57:18.32978Z',
 'endTimeUtc': '2021-10-03T22:09:38.370964Z',
 'services': {},
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'PCAP-Clus-DS12v2',
  'AMLSettingsJsonString': '{"path":null,"name":"PCAP_Exp_CarPrices_AutoML","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-159929","workspace_name":"quick-starts-ws-159929","region":"southcentralus","compute_target":"PCAP-Clus-DS12v2","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iter

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.run.automlrun?view=azure-ml-py#get-output-iteration--typing-union-int--nonetype----none--metric--typing-union-str--nonetype----none--return-onnx-model--bool---false--return-split-onnx-model--typing-union-azureml-automl-core-onnx-convert-onnx-convert-constants-splitonnxmodelname--nonetype----none----kwargs--typing-any-----typing-tuple-azureml-core-run-run--typing-any-
# If I decide to do the onnx models, use the get_output option of the automlrun in the link above to explain how

remote_run.wait_for_completion()
best_run, fitted_model = remote_run.get_output()
print("Details of best run")
print(best_run)
print("Fitted model")
print(fitted_model)
print("Steps of fitted model")
print(fitted_model.steps)

from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()   
print("Details of fitted model")
print_model(fitted_model)

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# best_run, fitted_model = automl_run.get_output()
# print(best_run)
# print(fitted_model)
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/nyc-taxi-data-regression-model-building.ipynb
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.ipynb
# https://docs.microsoft.com/en-us/azure/machine-learning/concept-onnx
# https://docs.microsoft.com/en-us/azure/machine-learning/concept-automated-ml#automl--onnx
# https://github.com/onnx/tutorials

Details of best run
Run(Experiment: PCAP_Exp_CarPrices_AutoML,
Id: AutoML_2531e0e0-ceba-4864-b453-a94fe7cc18c5_32,
Type: azureml.scriptrun,
Status: Completed)
Fitted model
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
), random_state=None))], verbose=False))], meta_learner=ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True, l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=None, normalize=False, positive=False, precompute='auto', random_state=None, selection='cyclic', tol=0.0001, verbose=0), training_cv_folds=5))],
                                     verbose=False

In [57]:

from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(best_run)

raw_explanations = client.download_model_explanation(raw=True)
print("Explanations of raw feature importance")
print(raw_explanations.get_feature_importance_dict())

engineered_explanations = client.download_model_explanation(raw=False)
print("Explanations of engineered feature importance")
print(engineered_explanations .get_feature_importance_dict())

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Explanations of raw feature importance
{'make': 5231.208014009412, 'year': 4586.179355115348, 'engine_size': 3641.9943364339865, 'miles': 2179.420158360328, 'model': 2019.1348045797881, 'body_type': 1765.052619310363, 'drivetrain': 1652.5170506806865, 'trim': 1408.8263900614154, 'fuel_type': 1163.86818154396, 'state': 834.937001754856, 'engine_block': 586.9135334878605, 'city': 150.75619377361465, 'zip': 149.425395406133, 'seller_name': 139.4658698779743, 'vehicle_type': 136.95199544572412, 'transmission': 60.93915061421192, 'street': 56.7292710620626, 'stock_no': 0.0, 'vin': 0.0, 'id': 0.0}
Explanations of imputed feature importance
{'year_MeanImputer': 4586.179355115616, 'engine_size_MeanImputer': 3449.7551830475495, 'miles_MeanImputer': 2189.6278557600394, 'drivetrain_CharGramCountVectorizer_fwd': 871.4660139863786, 'drivetrain_CharGramCountVectorizer_4wd': 796.6858296252691, 'make_CharGramCountVectorizer_nissan': 615.6758580467321, 'engine_size_ImputationMarker': 599.9917064687519,

In [45]:
#TODO: Save the best model
bestAutoname = best_run.properties['model_name']
bestAutofolder="model"
bestAutofolder2 = 'BestAutoMlModel'
downloadtarget = f'./{bestAutofolder}/{bestAutofolder2}'
print(f"Saving best model with name: {bestAutoname} to: {downloadtarget}")
model = remote_run.register_model(model_name = bestAutoname, description = 'AutoML-AutoPrices')
model.download(target_dir=downloadtarget, exist_ok = True)


Saving best model with name: AutoML2531e0e0c32 to: ./model/BestAutoMlModel


'model/BestAutoMlModel/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [1]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies
# If I decide to use onnx model
# https://docs.microsoft.com/en-us/azure/machine-learning/concept-automated-ml#automl--onnx

# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/resource-curated-environments.md
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-machine-learning-interpretability-automl
azureml_pip_packages = ['azureml-train-automl', 'azureml-defaults']
myenv_inference = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas', 'numpy', 'py-xgboost<=0.80'],
                                 pip_packages=azureml_pip_packages,
                                 pin_sdk_version=True)
with open("myenv_dependencies_automl.yml","w") as f:
    f.write(myenv_inference.serialize_to_string())
myenv_inference = Environment.from_conda_specification(name="myenv_inference", file_path="myenv_dependencies_automl.yml")

# myenv_inference = Environment.get(workspace=ws, name="AzureML-AutoML")

# myenv_inference = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
# conda_dep = CondaDependencies()
# conda_dep.add_pip_package("azureml-train-automl")
# conda_dep.add_pip_package("azureml-defaults")
# myenv_inference.python.conda_dependencies=conda_dep


print(f"Model ID - {model.id}")


# Combine scoring script & environment in Inference configuration
dummy_inference_config = InferenceConfig(entry_script="score_automl.py", environment=myenv_inference)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)

wsvc_name = "svc-autoprices-automl"
service = Model.deploy(
    ws,
    wsvc_name,
    [model],
    inference_config = dummy_inference_config,
    deployment_config = deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)
primary, secondary = service.get_keys()
print(primary)
#Enable app insights
service.update(enable_app_insights=True)

NameError: name 'ws' is not defined

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json
# TODO Download swagger.json and run on local docker image to review inputs.
# TODO Screenshot the input dict, outputs etc
service = Webservice(workspace=ws, name=wsvc_name)


import requests
# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back

#Generally, I figure we would send in all the columns from the original dataset and 
#then run it through the clean function to whittle it down, but that's not necessary for this exercise.
data = {
    "data":
    [
        {
            "id":"2fc65749-0cff",
            "vin":"1FTEW1E50JFB75149",
            "miles":"51611",
            "stock_no":"2021128A",
            "year":"2018",
            "make":"Ford",
            "model":"F-150",
            "trim":"XLT",
            "body_type":"Pickup",
            "vehicle_type":"Truck",
            "drivetrain":"4WD",
            "transmission":"Automatic",
            "fuel_type":"Unleaded / Unleaded",
            "engine_size":"5",
            "engine_block":"V",
            "seller_name":"holiday ford lincoln ltd",
            "street":"1555 Lansdowne Street, Peterborough",
            "city":"Peterborough",
            "state":"ON",
            "zip":"K9J 7M3"

            #Should be around 44699
        }
    ,
        {
            "id":"cd8fc59c-4ff5",
            "vin":"2T1BE40E39C017603",
            "miles":"335361",
            "stock_no":"1385",
            "year":"2009",
            "make":"Toyota",
            "model":"Corolla",
            "trim":"XRS",
            "body_type":"Sedan",
            "vehicle_type":"Car",
            "drivetrain":"FWD",
            "transmission":"Manual",
            "fuel_type":"Unleaded",
            "engine_size":"2.4",
            "engine_block":"I",
            "seller_name":"fara auto sales",
            "street":"1545 Warden Ave",
            "city":"Scarborough",
            "state":"ON",
            "zip":"M1R 2S9"

            #Should be around 3550
        }
    ,
        {
            "id":"77521bd9-e533",
            "vin":"W1N0G6EB4LV247708",
            "miles":"8852",
            "stock_no":"M21144A",
            "year":"2020",
            "make":"Mercedes-Benz",
            "model":"GLC-Class",
            "trim":"AMG GLC43",
            "body_type":"SUV",
            "vehicle_type":"Truck",
            "drivetrain":"4WD",
            "transmission":"Automatic",
            "fuel_type":"Premium Unleaded",
            "engine_size":"3",
            "engine_block":"V",
            "seller_name":"kelowna mercedes-benz",
            "street":"2580 Enterprise Way",
            "city":"Kelowna",
            "state":"BC",
            "zip":"V1X 7X5"

            #Should be around 79995
        }
    ]
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics

print(service.get_logs())

service.delete()

In [ ]:
print('x')